<a href="https://colab.research.google.com/github/PrathyushaRagavAdari/CS5588_DS_Capstone/blob/main/Week_2_Hands_On_with_Advanced_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community faiss-cpu sentence-transformers openai tiktoken

In [2]:
# Create a dummy Policy Document (The "RAG" part)
policy_text = """
Security Policy 101:
1. Low Risk (Score < 30): Verify using email confirmation only.
2. Medium Risk (Score 30-70): Challenge user with a question about their last transaction date.
3. High Risk (Score > 70): Challenge user to provide the exact amount of their last purchase and their registered pet's name.
4. Flagged Accounts: Immediately suspend and refer to human agent.
"""

# Create a dummy User Database (The "Context" part)
user_db = {
    "john_doe": {
        "risk_score": 75,
        "last_transaction": "2023-10-12",
        "last_amount": "$45.50",
        "pet_name": "Rover"
    }
}

In [3]:
!pip install langchain-huggingface faiss-cpu sentence-transformers

import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

policy_text = """
VERIFICATION POLICY MANUAL - REV 2026
-------------------------------------
1. LOW RISK (Score 0-30):
   - Protocol: "Passive Verification"
   - Action: Send a 6-digit code to the registered email. Do not interrupt user flow.

2. MEDIUM RISK (Score 31-70):
   - Protocol: "Knowledge Challenge"
   - Action: Ask the user to verify the date of their last transaction.
   - Governance: If the user gets it wrong once, lock account for 1 hour.

3. HIGH RISK (Score 71-99):
   - Protocol: "Biometric & Knowledge Depth"
   - Action: Require TWO factors:
     1. The exact dollar amount of the last transaction.
     2. The name of the registered pet.
   - Governance: Any failure results in immediate account suspension.

4. CRITICAL/UNKNOWN (Score 100+):
   - Protocol: "Human Handoff"
   - Action: Do not attempt automated verification. Output "MANUAL_REVIEW_REQUIRED".
"""

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents([policy_text])

print("Building Vector Database... (This runs locally on Colab CPU)")
# 'all-MiniLM-L6-v2' is a small, fast model that downloads once
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(docs, embeddings)
retriever = vector_db.as_retriever(search_kwargs={"k": 1})

print("✅ Vector Database Built Successfully.")

Building Vector Database... (This runs locally on Colab CPU)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Vector Database Built Successfully.


In [4]:
!pip install -q transformers accelerate bitsandbytes langchain-huggingface

import torch
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

print("🔄 Loading Local 'Flan-T5-Large' Model... (This uses Colab RAM, not API)")

model_id = "google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.1,
    do_sample=True
)

llm = HuggingFacePipeline(pipeline=pipe)
print(f"✅ Success: {model_id} is loaded and ready.")

template = """
You are a Security Agent. strictly follow the policy below.

POLICY:
{policy_context}

CURRENT USER STATUS:
Risk Score: {risk_score}
User Profile: {user_profile}

INSTRUCTIONS:
1. Check the policy for the given risk score.
2. Write the specific verification challenge question for this user.
3. If the policy says manual review, write exactly: "MANUAL_REVIEW_REQUIRED".

YOUR RESPONSE:
"""

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {
        "policy_context": itemgetter("risk_query") | retriever | format_docs,
        "user_profile": itemgetter("user_profile"),
        "risk_score": itemgetter("risk_score")
    }
    | prompt
    | llm
    | StrOutputParser()
)

current_user_context = {
    "username": "j_doe",
    "risk_score": 75,
    "last_transaction_date": "2025-10-12",
    "last_amount": "$45.50",
    "pet_name": "Rover"
}

print(f"🤖 Processing User: {current_user_context['username']} (Risk: {current_user_context['risk_score']})")
print("-" * 40)

response = chain.invoke({
    "risk_query": f"What is the protocol for risk score {current_user_context['risk_score']}?",
    "user_profile": str(current_user_context),
    "risk_score": str(current_user_context['risk_score'])
})

print("Agent Output:\n" + response)

🔄 Loading Local 'Flan-T5-Large' Model... (This uses Colab RAM, not API)


Device set to use cpu


✅ Success: google/flan-t5-large is loaded and ready.
🤖 Processing User: j_doe (Risk: 75)
----------------------------------------
Agent Output:
'username': 'j_doe', 'risk_score': 75, 'last_transaction_date': '2025-10-12', 'last_amount': '$45.50', 'pet_name': 'Rover'


In [5]:
#GOVERNANCE LAYER (The Safety Guardrail)

def secure_identity_gateway(user_context):
    """
    This function acts as the 'Governance Layer'.
    It intercepts the AI's response before the user sees it.
    """

    # 1. Run the Agent
    # We construct the query dynamically based on risk score
    query_text = f"What is the protocol for risk score {user_context['risk_score']}?"

    raw_response = chain.invoke({
        "risk_query": query_text,
        "user_profile": str(user_context),
        "risk_score": str(user_context['risk_score'])
    })

    # 2. The "Safety Check" (Governance)
    # If the model flags a critical risk, we override the output to a system lock.
    if "MANUAL_REVIEW_REQUIRED" in raw_response:
        return {
            "status": "🔴 BLOCKED",
            "action": "ACCOUNT LOCKED. Ticket #9928 created for Human Review.",
            "raw_log": raw_response
        }

    # 3. If safe, pass the challenge through
    return {
        "status": "🟢 ACTIVE",
        "action": f"Challenge Issued: {raw_response}",
        "raw_log": raw_response
    }

# --- TEST THE GOVERNANCE LAYER ---

# Case A: Normal User (Risk 75)
print("Testing Normal User...")
user_safe = {"username": "alice", "risk_score": 75, "user_profile": "Standard User"}
print(secure_identity_gateway(user_safe))
print("\n" + "-"*30 + "\n")

# Case B: Critical User (Risk 150) -> Should Trigger Lock
print("Testing Critical Risk User...")
user_risky = {"username": "hacker_bot", "risk_score": 150, "user_profile": "Unknown Device"}
print(secure_identity_gateway(user_risky))

Testing Normal User...
{'status': '🟢 ACTIVE', 'action': "Challenge Issued: 'username': 'alice', 'risk_score': 75, 'user_profile': 'Standard User'", 'raw_log': "'username': 'alice', 'risk_score': 75, 'user_profile': 'Standard User'"}

------------------------------

Testing Critical Risk User...
{'status': '🟢 ACTIVE', 'action': "Challenge Issued: 'username': 'hacker_bot', 'risk_score': 150, 'user_profile': 'Unknown Device'", 'raw_log': "'username': 'hacker_bot', 'risk_score': 150, 'user_profile': 'Unknown Device'"}


In [6]:
# FAILURE SIMULATION & SYSTEM FIX

# 1. THE FAILURE
# Scenario: A new threat type "Deepfake" appears, but it's not in our policy text.
print("💥 SIMULATING FAILURE: Unknown Threat Type")
unknown_context = {"username": "fake_video", "risk_score": 90, "notes": "Deepfake Detected"}

# The agent will likely struggle or give the wrong rule because "Deepfake" isn't in the DB.
response = chain.invoke({
    "risk_query": "Protocol for Deepfake Detected?",
    "user_profile": str(unknown_context),
    "risk_score": "90"
})
print(f"Agent Output (Likely Wrong/Generic): {response}")


# 2. THE SYSTEM FIX (Data Update)
# We update the policy to explicitly handle this new threat.
print("\n🛠️ APPLYING FIX: Updating Policy & Re-indexing...")

new_policy_rule = """
5. DEEPFAKE THREAT (Any Score):
   - Protocol: "Liveness Failure"
   - Action: Do not challenge. Auto-ban user UUID immediately.
"""

# Append and Re-process
updated_policy = policy_text + "\n" + new_policy_rule
new_docs = text_splitter.create_documents([updated_policy])
vector_db = FAISS.from_documents(new_docs, embeddings) # Rebuild the brain
retriever = vector_db.as_retriever(search_kwargs={"k": 1})

# Re-build the chain with the new retriever
chain = (
    {
        "policy_context": itemgetter("risk_query") | retriever | format_docs,
        "user_profile": itemgetter("user_profile"),
        "risk_score": itemgetter("risk_score")
    }
    | prompt
    | llm
    | StrOutputParser()
)

# 3. VERIFY THE FIX
print("\n✅ VERIFYING FIX:")
response_fixed = chain.invoke({
    "risk_query": "Protocol for Deepfake Detected?",
    "user_profile": str(unknown_context),
    "risk_score": "90"
})
print(f"New Agent Output: {response_fixed}")

💥 SIMULATING FAILURE: Unknown Threat Type
Agent Output (Likely Wrong/Generic): 'username': 'fake_video', 'risk_score': 90, 'notes': 'Deepfake Detected'

🛠️ APPLYING FIX: Updating Policy & Re-indexing...

✅ VERIFYING FIX:
New Agent Output: 'username': 'fake_video', 'risk_score': 90, 'notes': 'Deepfake Detected'


In [10]:
!pip install -q rank_bm25 langchain langchain-community

from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import sys

try:
    from langchain.retrievers import EnsembleRetriever
    print("✅ Loaded EnsembleRetriever from langchain package.")
except ImportError:
    print("⚠️ Import failed. Using local 'EnsembleRetriever' fallback to avoid restart.")
    from langchain_core.retrievers import BaseRetriever
    from langchain_core.documents import Document
    from typing import List

    class EnsembleRetriever(BaseRetriever):
        retrievers: List[BaseRetriever]
        weights: List[float]

        def _get_relevant_documents(self, query: str, *, run_manager=None) -> List[Document]:
            all_docs = []
            for r in self.retrievers:
                all_docs.extend(r.invoke(query))
            seen = set()
            unique_docs = []
            for d in all_docs:
                if d.page_content not in seen:
                    unique_docs.append(d)
                    seen.add(d.page_content)
            return unique_docs[:4]
# --- END FIX ---

print("✅ Libraries imported successfully.")

# 1. SETUP THE TARGET DOCUMENTS
if 'new_docs' in globals():
    print("ℹ️ Using Updated Policy Documents (Deepfake rule included).")
    target_docs = new_docs
elif 'docs' in globals():
    print("⚠️ Using Original Policy Documents (Deepfake rule might be missing).")
    target_docs = docs
else:
    raise ValueError("❌ No documents found! Please run the 'Vector Database' cell first.")

# 2. RE-INITIALIZE EMBEDDINGS
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 3. BUILD VECTOR RETRIEVER
print("1️⃣ Building Vector Retriever...")
vector_db = FAISS.from_documents(target_docs, embeddings)
vector_retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# 4. BUILD KEYWORD RETRIEVER
print("2️⃣ Building Keyword Retriever (BM25)...")
keyword_retriever = BM25Retriever.from_documents(target_docs)
keyword_retriever.k = 2

# 5. BUILD HYBRID RETRIEVER
print("3️⃣ Assembling Hybrid Retriever (Real Weighted Version)...")
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, keyword_retriever],
    weights=[0.5, 0.5]
)

print("✅ Success: Hybrid Retriever is ready.")

⚠️ Import failed. Using local 'EnsembleRetriever' fallback to avoid restart.
✅ Libraries imported successfully.
ℹ️ Using Updated Policy Documents (Deepfake rule included).


/tmp/ipython-input-3650594660.py:50: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


1️⃣ Building Vector Retriever...
2️⃣ Building Keyword Retriever (BM25)...
3️⃣ Assembling Hybrid Retriever (Real Weighted Version)...
✅ Success: Hybrid Retriever is ready.


In [11]:
# EVALUATION METRICS (Precision & Trust)

def evaluate_system():
    test_set = [
        {"query": "Score 20", "expected": "Passive Verification"},
        {"query": "Score 75", "expected": "Biometric"},
        {"query": "Score 150", "expected": "MANUAL_REVIEW"}
    ]

    correct_retrievals = 0
    total_trust_score = 0

    print("\n📊 RUNNING SYSTEM EVALUATION...")
    print("-" * 40)

    for case in test_set:
        retrieved_docs = ensemble_retriever.invoke(case["query"])
        retrieved_text = retrieved_docs[0].page_content
        print(f"🔍 Query: '{case['query']}'")
        if case["expected"] in retrieved_text:
            correct_retrievals += 1
            print(f"✅ Query '{case['query']}' -> Found correct policy.")
            total_trust_score += 5 # 5/5 Trust
        else:
            print(f"❌ Query '{case['query']}' -> Missed context.")
            total_trust_score += 1 # 1/5 Trust

    precision = correct_retrievals / len(test_set)
    avg_trust = total_trust_score / len(test_set)

    print("-" * 40)
    print(f"📈 TECHNICAL METRIC: Precision@1 = {precision:.2f}")
    print(f"🤝 PRODUCT METRIC: Avg Trust Score = {avg_trust:.1f}/5.0")

    return {"precision": precision, "trust_score": avg_trust}

metrics = evaluate_system()


📊 RUNNING SYSTEM EVALUATION...
----------------------------------------
🔍 Query: 'Score 20'
❌ Query 'Score 20' -> Missed context.
🔍 Query: 'Score 75'
✅ Query 'Score 75' -> Found correct policy.
🔍 Query: 'Score 150'
✅ Query 'Score 150' -> Found correct policy.
----------------------------------------
📈 TECHNICAL METRIC: Precision@1 = 0.67
🤝 PRODUCT METRIC: Avg Trust Score = 3.7/5.0


In [12]:
# UPDATE AGENT TO USE HYBRID RETRIEVER

from langchain_core.runnables import RunnablePassthrough


chain = (
    {
        "policy_context": itemgetter("risk_query") | ensemble_retriever | format_docs,
        "user_profile": itemgetter("user_profile"),
        "risk_score": itemgetter("risk_score")
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Agent is now connected to the Hybrid Brain (BM25 + Vectors).")

✅ Agent is now connected to the Hybrid Brain (BM25 + Vectors).
